In [18]:
import pandas as pd
df = pd.read_csv(r"C:\Users\jairo\OneDrive - Pontificia Universidad Javeriana\Desktop\SUPPORT-VECTOR-MACHINE-CLASSIFIER\Data\03 CSV data -- STC(A)_numerical dates.csv")

# PREPARACIÓN DE DATOS

In [5]:
# Librerías para manipulación y preparación del dataframe

import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [6]:
# Se elimina ID porque es un identificador único y no aporta información al modelo

df = df.drop(columns=["ID"], errors="ignore")

In [7]:
# Se detectan automáticamente columnas que contienen "Date"

date_cols = [col for col in df.columns if "Date" in col]

# Conversión a datetime
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

# Se crea una referencia temporal para transformar fechas a números
reference_date = df[date_cols].min().min()

# Se transforman a "días desde la fecha mínima"
for col in date_cols:
    df[col + "_days"] = (df[col] - reference_date).dt.days

# Se eliminan las columnas originales de fecha
df = df.drop(columns=date_cols)

In [8]:
# Convierte columnas tipo object a numéricas cuando es posible

for col in df.select_dtypes(include="object").columns:
    try:
        df[col] = pd.to_numeric(df[col])
    except:
        pass

In [9]:
# Se detecta automáticamente la columna objetivo que contiene "Retained"

target_col = [col for col in df.columns if "Retained" in col][0]

y = pd.to_numeric(df[target_col], errors="coerce")

X = df.drop(columns=[target_col])

In [10]:
# Separación automática por tipo de dato

num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object", "category"]).columns

In [11]:
# Agrupa categorías poco frecuentes en "OTHER"
# Evita expansión excesiva en el OneHotEncoding

max_categories = 20

for col in cat_cols:
    top = X[col].value_counts().nlargest(max_categories).index
    X[col] = np.where(X[col].isin(top), X[col], "OTHER")

In [12]:
# Para numéricas:
# imputación con mediana (robusta a outliers)
# escalado obligatorio para SVM

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

In [13]:
# Para categóricas:
# imputación de missing como categoría
# OneHotEncoding para convertir a variables binarias

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="Missing")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

In [14]:
# Aplica cada pipeline según el tipo de columna

preprocess = ColumnTransformer([
    ("num", num_pipeline, num_cols),
    ("cat", cat_pipeline, cat_cols)
])

In [15]:
# Ajusta el preprocesador y transforma los datos
# El resultado es una matriz numérica lista para SVM

X_prepared = preprocess.fit_transform(X)

In [16]:
# Se obtienen los nombres de las columnas generadas por el preprocesamiento
feature_names = preprocess.get_feature_names_out()

# Se convierte a matriz densa si el resultado es sparse
X_prepared_dense = X_prepared.toarray() if hasattr(X_prepared, "toarray") else X_prepared

# Se construye el DataFrame final listo para el modelo
X_prepared = pd.DataFrame(
    X_prepared_dense,
    columns=feature_names,
    index=X.index
)

In [17]:
#ESTOS SON LOS DATOS LISTOS PARA EL SVM
X_prepared
y

0       1
1       1
2       1
3       0
4       0
       ..
2384    0
2385    1
2386    1
2387    1
2388    1
Name: Retained.in.2012., Length: 2389, dtype: int64